In [6]:
import pandas as pd
import numpy as np
from PIL import Image
import re

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [8]:
%config Completer.use_jedi = False

In [1]:
from transformers import AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
tokenizer.tokenize("안녕 나는 나는나나나")

['안녕', '나', '##는', '나', '##는', '##나', '##나', '##나']

In [2]:
import pandas as pd
original_train_df = pd.read_csv('/opt/ml/input/data/train/train.tsv', delimiter='\t', names=['id', 'text', 'entity1', 'beg1', 'end1', 'entity2', 'beg2', 'end2', 'label'])


In [7]:
original_train_df.head()

,id,text,entity1,beg1,end1,entity2,beg2,end2,label
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [8]:
original_train_df.iloc[[0, 1, 2]]

,id,text,entity1,beg1,end1,entity2,beg2,end2,label
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭


In [5]:
import pandas as pd
original_train_df = pd.read_csv('/opt/ml/input/data/train/train.tsv', delimiter='\t', names=['id', 'text', 'entity1', 'beg1', 'end1', 'entity2', 'beg2', 'end2', 'label'])


In [1]:
def load_pd_data(data_dir):
    """
    학습 데이터를 pandas DataFrmae 형태로 읽어들이고, label을 id로 변환한다
    - data_dir: train tsv 파일의 경로
    """
    with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
        label_type = pickle.load(f)
        print(label_type)
    # load dataset
    dataset = pd.read_csv(data_dir, delimiter='\t', names=["documentID", "text", "ent1", "ent1_start", "ent1_end", "ent2", "ent2_start", "ent2_end", "label"])
    label = []
    for i in dataset['label']:
        if i == 'blind':
            label.append(100)
        else:
            label.append(label_type[i])
    dataset['label'] = label
    return dataset

In [15]:
from sklearn.model_selection import StratifiedKFold
import pickle
  
stratified_kfolds = StratifiedKFold(n_splits=5)

dataset = load_pd_data("/opt/ml/input/data/train/train.tsv")
labels = dataset['label'].values

for train_idx, val_idx in stratified_kfolds.split(dataset, labels):
  print(train_idx, val_idx)
  train_dataset = dataset.iloc[train_idx]
  train_labels = train_dataset['label']
  val_dataset = dataset.iloc[val_idx]
  val_labels = val_dataset['label']
  tokenized_train = tokenize_dataset(train_dataset, tokenizer)
  tokenized_val = tokenize_dataset(val_dataset, tokenizer)
  train_dataset = RE_Dataset(tokenized_train, train_labels)
  val_dataset = RE_Dataset(tokenized_val, val_labels)
  train_model(args, train_dataset, val_dataset, model_output_dir)

[ 467  883 1137 ... 8997 8998 8999] [   0    1    2 ... 3599 3697 5211]
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


NameError: name 'tokenize_dataset' is not defined

In [6]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
tokenized_lengths = []
for text in original_train_df['text']:
    tokenized_lengths.append(len(tokenizer.encode(text)))
original_train_df['tokenized_length'] = tokenized_lengths

In [7]:
original_train_df['tokenized_length'].describe()

count    9000.000000
mean       59.806111
std        29.070114
min        13.000000
25%        40.000000
50%        54.000000
75%        72.000000
max       341.000000
Name: tokenized_length, dtype: float64

In [14]:
len(original_train_df[original_train_df["tokenized_length"] > 120])

216

In [221]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

def load_pd_data_2(data_dir):
    """
    학습 데이터를 pandas DataFrmae 형태로 읽어들이고, label을 id로 변환한다
    - data_dir: train tsv 파일의 경로
    """
    with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
        label_type = pickle.load(f)
        print(label_type)
    # load dataset
    dataset = pd.read_csv(data_dir, delimiter='\t')
    labels = []
    dataset["label"].value_counts()
    for label in dataset['label']:
        if label == 'blind':
            labels.append(100)
        else:
            labels.append(label_type[label])
    dataset['label'] = labels
    return dataset


train_data = load_pd_data_2("/opt/ml/input/data/train/train_gold.tsv")
labels = train_data['label'].values

x_train, x_val, y_train, y_val = train_test_split(train_data, labels, test_size=0.1, stratify=labels)



{'관계_없음': 0, '인물:배우자': 1, '인물:직업/직함': 2, '단체:모회사': 3, '인물:소속단체': 4, '인물:동료': 5, '단체:별칭': 6, '인물:출신성분/국적': 7, '인물:부모님': 8, '단체:본사_국가': 9, '단체:구성원': 10, '인물:기타_친족': 11, '단체:창립자': 12, '단체:주주': 13, '인물:사망_일시': 14, '단체:상위_단체': 15, '단체:본사_주(도)': 16, '단체:제작': 17, '인물:사망_원인': 18, '인물:출생_도시': 19, '단체:본사_도시': 20, '인물:자녀': 21, '인물:제작': 22, '단체:하위_단체': 23, '인물:별칭': 24, '인물:형제/자매/남매': 25, '인물:출생_국가': 26, '인물:출생_일시': 27, '단체:구성원_수': 28, '단체:자회사': 29, '인물:거주_주(도)': 30, '단체:해산일': 31, '인물:거주_도시': 32, '단체:창립일': 33, '인물:종교': 34, '인물:거주_국가': 35, '인물:용의자': 36, '인물:사망_도시': 37, '단체:정치/종교성향': 38, '인물:학교': 39, '인물:사망_국가': 40, '인물:나이': 41}


In [222]:
train_data["train_val"] = [0 for _ in range(len(train_data))]
train_data.head()

,id,text,entity1,beg1,end1,entity2,beg2,end2,label,train_val
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,17,0
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,0,0
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,6,0
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,2,0
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,8,0


In [227]:
train_data["train_val"].loc[x_val.index] = 1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [228]:
train_data["train_val"].value_counts()

0    8159
1     907
Name: train_val, dtype: int64

In [234]:
# train_data.loc[train_data["train_val"] == 1]
train_data.loc[train_data["label"] == 10].loc[train_data["train_val"] == 1]
train_data.to_csv("/opt/ml/input/data/train/train_gold_spilt.tsv", sep="\t", index=None)

In [236]:
read_data = pd.read_csv("/opt/ml/input/data/train/train_gold_spilt.tsv", delimiter="\t")
read_data.head()

,id,text,entity1,beg1,end1,entity2,beg2,end2,label,train_val
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,17,0
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,0,0
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,6,0
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,2,1
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,8,0


# Relabel data
는 inplace로 바꾸는 법을 까먹어서 수작업함..

**잘못된 데이터**

- wikitree-55837-4-0-2-10-11	단체:구성원
- wikitree-62775-3-3-7-0-2	단체:본사_도시
- wikitree-12599-4-108-111-4-7        관계_없음
- wikipedia-25967-115-24-26-35-37  관계_없음
- wikipedia-16427-6-14-17-20-22	 관계_없음
- wikipedia-16427-8-0-3-26-28	 관계_없음
- wikitree-19765-5-30-33-6-8              관계_없음
- wikitree-58702-0-18-20-22-24          관계_없음
- wikitree-71638-8-21-23-15-17          관계_없음


In [52]:
corrected_data = {
                    "wikitree-55837-4-0-2-10-11": "단체:구성원",
                    "wikitree-62775-3-3-7-0-2": "단체:본사_도시",
                    "wikitree-12599-4-108-111-4-7" : "관계_없음",
                    "wikipedia-25967-115-24-26-35-37" : "관계_없음",
                    "wikipedia-16427-6-14-17-20-22" : "관계_없음",
                    "wikipedia-16427-8-0-3-26-28" :	"관계_없음",
                    "wikitree-19765-5-30-33-6-8" : "관계_없음",
                    "wikitree-58702-0-18-20-22-24" : "관계_없음",
                    "wikitree-71638-8-21-23-15-17" : "관계_없음",
                  }


In [29]:
for wiki_id, corrected_val in corrected_data.items():
    original_train_df[original_train_df['wiki_id'] == "wiki_id"]["label"] = corrected_val
    print(corrected_val)

단체:구성원
단체:본사_도시
관계_없음
관계_없음
관계_없음
관계_없음
관계_없음
관계_없음
관계_없음


# Kor-re-gold 데이터셋 추가 사용

In [9]:
cd /opt/ml/input/data/new_data


/opt/ml/input/data/new_data


In [4]:
!git clone https://github.com/machinereading/kor-re-gold.git

Cloning into 'kor-re-gold'...
remote: Enumerating objects: 51, done.
remote: Total 51 (delta 0), reused 0 (delta 0), pack-reused 51
Unpacking objects: 100% (51/51), done.


In [10]:
cd /opt/ml/input/data/new_data/kor-re-gold

/opt/ml/input/data/new_data/kor-re-gold


In [170]:
new_train_df = pd.read_csv("./gold-standard-v1/agreement_content.txt", delimiter="\t", names=["entity1", "entity2", "label", "text", "id", "ans"])
new_train_df = new_train_df[(new_train_df["ans"] == "yes")]
print(len(new_train_df))
new_inag_train_df = pd.read_csv("./gold-standard-v1/conflict2agreement_content.txt", delimiter="\t", names=["entity1", "entity2", "label", "text", "id", "ans"])
new_inag_train_df = new_inag_train_df[(new_inag_train_df["ans"] == "yes")]
print(len(new_inag_train_df))

161
75


In [171]:
gold_to_ours = {
"spouse": '인물:배우자',
"occupation": '인물:직업/직함',
"industry": '단체:모회사', # 이건 주제인 회사 or 단체가 속한 산업 분야이니 다를듯
'party': '인물:소속단체', # 항목 주제인 정치인 또는 당원이 소속되어 있는 당 
'nationality': '인물:출신성분/국적', # -> 인간의 국적인지는 확실 X
'parent': '인물:부모님',
'currentMember': '단체:구성원',
'pastMember': '단체:구성원',
'leaderName': '단체:구성원',
'keyPerson': '단체:구성원',
'relative': '인물:기타_친족',
'foundedBy': '단체:창립자',
'parentCompany': '단체:상위_단체',
'headquarter': '단체:본사_주(도)', # 항목 주제인 조직이나 단체의 본거지가 있거나 세워져 있는 구체적인 위치 (위치라서.. 주(도)랑 별 차이 없을수도)
'manufacturer': '단체:제작', # 항목 주제을(를) 제조/제작한 주요 회사, product, industry
'product': '단체:제작', 
'industry': '단체:제작',
'child': '인물:자녀',
'producer': '인물:제작', # 항목 주제인 창작물의 제작자
'writer': '인물:제작',
'artist': '인물:제작',
'author': '인물:제작',
'composer': '인물:제작',
'director': '인물:제작',
'channels': '인물:제작',
'musicalArtist': '인물:제작',
'creativeDirector': '인물:제작',
'country': '인물:출생_국가', # 항목 주제의 국가 # 잘 안맞는듯
# '단체:자회사': 29, # parentCompany?
# 'religion': '인물:종교', # 주의: 단체와 연관된 경우도 있음
'deathPlace': '인물:사망_국가', # deathPlace - 항목 주제가 죽은 장소,, 지만 도시와 국가가 구분되지 않음. 하지만 시도했다
'ideology': '단체:정치/종교성향', # ideology - 항목주제인 기관의 정치적 이념
'education': '인물:학교', # 항목 주제인 인물이 교육받은 곳
}

In [172]:
new_label = [gold_to_ours[old_label] if old_label in gold_to_ours else 'None' for old_label in new_train_df["label"]]

In [173]:
new_train_df["label"] = new_label

In [174]:
new_train_df["label"].value_counts()

None          95
인물:제작         18
단체:구성원        10
단체:제작          7
인물:출생_국가       5
인물:자녀          3
인물:배우자         3
단체:정치/종교성향     3
인물:기타_친족       3
인물:사망_국가       3
단체:상위_단체       2
인물:부모님         2
단체:창립자         2
인물:직업/직함       2
인물:출신성분/국적     2
인물:소속단체        1
Name: label, dtype: int64

In [175]:
new_train_df = new_train_df.loc[new_train_df["label"] != 'None']
new_train_df["label"].value_counts()

인물:제작         18
단체:구성원        10
단체:제작          7
인물:출생_국가       5
인물:자녀          3
인물:배우자         3
단체:정치/종교성향     3
인물:기타_친족       3
인물:사망_국가       3
단체:상위_단체       2
인물:부모님         2
단체:창립자         2
인물:직업/직함       2
인물:출신성분/국적     2
인물:소속단체        1
Name: label, dtype: int64

In [176]:
import re

beg1 = []
beg2 = []
end1 = []
end2 = []
texts = []

for txt, ent1, ent2 in zip(new_train_df["text"], new_train_df["entity1"], new_train_df["entity2"]):
# for txt in list(new_train_df["text"]):
    match1 = re.search("\[\[ \_sbj\_ \]\]", txt)
    match2 = re.search("\[\[ \_obj\_ \]\]", txt)
    ent1_start, ent1_end = match1.span()
    ent2_start, ent2_end = match2.span()
    print(txt)
    print(match1.span())
    print(match2.span())
    diff_len_2 = len(ent2) - 11
    diff_len_1 = len(ent1) - 11
    if ent1_start > ent2_start:
        txt = txt[:ent2_start] + ent2 + txt[ent2_end:]
        ent2_end += diff_len_2
        ent1_start += diff_len_2
        txt = txt[:ent1_start] + ent1 + txt[ent1_start+11:]
        ent1_end = ent1_start + len(ent1)

    else:
        txt = txt[:ent1_start] + ent1 + txt[ent1_start+11:]
        ent1_end += diff_len_1
        ent2_start += diff_len_1
        txt = txt[:ent2_start] + ent2 + txt[ent2_end:]
        ent2_end = ent2_start + len(ent2)
    print(txt[ent1_start:ent1_end])
    print(txt[ent2_start:ent2_end])
    print(txt)
    texts.append(txt)
    beg1.append(ent1_start)
    beg2.append(ent2_start)
    end1.append(ent1_end - 1)
    end2.append(ent2_end - 1)
new_train_df["text"] = texts
new_train_df["beg1"] = beg1
new_train_df["beg2"] = beg2
new_train_df["end1"] = end1
new_train_df["end2"] = end2



초대 준남작, 제임스 니콜라스 서덜랜드 매터슨(, ,  [[ 1796년 ]]   [[ 11월_17일 ]]  -  [[ 1878년 ]]   [[ 12월_31일 ]] )은  [[ 영국 ]]   [[ 스코틀랜드 ]] 의 사업가로  [[ _obj_ ]] 과 함께  [[ _sbj_ ]]  상회를 설립했다.
(144, 155)
(126, 137)
자딘_매터슨
윌리엄_자딘
초대 준남작, 제임스 니콜라스 서덜랜드 매터슨(, ,  [[ 1796년 ]]   [[ 11월_17일 ]]  -  [[ 1878년 ]]   [[ 12월_31일 ]] )은  [[ 영국 ]]   [[ 스코틀랜드 ]] 의 사업가로  윌리엄_자딘 과 함께  자딘_매터슨  상회를 설립했다.
 미국 최초의 흑인  [[ 퍼스트_레이디 ]]   [[ 미셸_오바마 ]] , 전 부통령  [[ 에런_버 ]] , 전 국방부 장관  [[ 도널드_럼즈펠드 ]] , 전 국무장관  [[ 존_포스터_덜레스 ]] , '봉쇄정책의 아버지'로 알려진 외교관  [[ 조지_케넌 ]] ,  [[ 미국_연방대법원 ]]  대법관  [[ 소니아_소토마요르 ]] 와  [[ 엘레나_케이건 ]] , 《 [[ 포브스 ]] 》지 발행인  [[ 스티브_포브스 ]] ,  [[ 구글 ]] 의 CEO  [[ 에릭_슈밋 ]] ,  [[ 이베이 ]] 의 CEO  [[ 메그_휘트먼 ]] ,  [[ _sbj_ ]]  설립자  [[ _obj_ ]] ,  [[ 노벨_물리학상 ]]   [[ 리처드_파인만 ]] , '컴퓨터 과학의 아버지'  [[ 앨런_튜링 ]] ,  [[ 필즈상 ]] 을 수상한 수학자  [[ 테렌스_타오 ]] ,  [[ 존_밀너 ]] , 만줄 바르가바와 물리학자  [[ 에드워드_위튼 ]] ,  [[ 아벨상 ]] 을 수상한 수학자  [[ 존_테이트 ]] ,  [[ 야코프_그리고리예비치_시나이 ]] 도 프린스턴을 졸업했다.
(310, 321)
(328, 339)
아마존닷컴
제프_베조스
 미국 최초의 흑인  [[ 퍼스트_레이디 ]]   [[ 미셸_오바마 ]] , 전 부

In [178]:
new_train_df.drop(columns=["ans"], inplace=True)
new_train_df.head()
new_train_df.to_csv("/opt/ml/input/data/train/gold_dataset.tsv", sep="\t", index=False)

In [164]:
original_train_df = pd.read_csv('/opt/ml/input/data/train/train.tsv', delimiter='\t', names=['id', 'text', 'entity1', 'beg1', 'end1', 'entity2', 'beg2', 'end2', 'label'])
original_train_df.head()

,id,text,entity1,beg1,end1,entity2,beg2,end2,label
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [179]:
gold_train_df = pd.read_csv("/opt/ml/input/data/train/gold_dataset.tsv", delimiter="\t")
gold_train_df.head()

,entity1,entity2,label,text,id,beg1,beg2,end1,end2
0,자딘_매터슨,윌리엄_자딘,단체:창립자,"초대 준남작, 제임스 니콜라스 서덜랜드 매터슨(, , [[ 1796년 ]] [...",foundedBy378,139,126,144,131
1,아마존닷컴,제프_베조스,단체:구성원,"미국 최초의 흑인 [[ 퍼스트_레이디 ]] [[ 미셸_오바마 ]] , 전 부...",keyPerson373,310,322,314,327
2,좀비랜드,폴_워닉,인물:제작,"폴_워닉 과 함께 쓴 《 좀비랜드 》(2009), 《 [[ 지.아이.조_2 ]]...",writer375,16,2,19,5
3,북송,북송_휘종,단체:구성원,정화(政和)는 북송 의 휘종(徽宗)인 북송_휘종던 [[ 연호 ]] 이다.,leaderName245,9,23,10,27
4,N.W.A,아이스_큐브,단체:구성원,"[[ 1992년 ]] , 자신이 존경했던 힙합 그룹 N.W.A 의 멤버인 아...",pastMember254,32,45,36,50


In [182]:
beg1 = original_train_df["beg1"][0]
beg2 = original_train_df["beg2"][0]
end1 = original_train_df["end1"][0]
end2 = original_train_df["end2"][0]
txt = original_train_df["text"][0]
print(txt[beg1:end1+1], txt[beg2:end2+1])

beg1 = gold_train_df["beg1"][0]
beg2 = gold_train_df["beg2"][0]
end1 = gold_train_df["end1"][0]
end2 = gold_train_df["end2"][0]
txt = gold_train_df["text"][0]
print(txt[beg1:end1+1], txt[beg2:end2+1])


랜드로버 자동차
자딘_매터슨 윌리엄_자딘


In [190]:
combined_df = pd.concat([original_train_df, gold_train_df], ignore_index=True)
combined_df.head()

,id,text,entity1,beg1,end1,entity2,beg2,end2,label
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [191]:
combined_df["label"].value_counts()

관계_없음          4438
단체:구성원          825
인물:소속단체         680
인물:직업/직함        555
단체:상위_단체        337
단체:별칭           231
인물:출신성분/국적      211
인물:동료           186
단체:제작           176
인물:부모님          166
단체:본사_도시        136
단체:본사_국가        115
인물:별칭           103
인물:배우자           95
인물:자녀            87
단체:하위_단체         79
인물:제작            74
단체:창립일           67
인물:기타_친족         61
인물:형제/자매/남매      52
단체:창립자           47
인물:사망_일시         45
단체:모회사           44
인물:출생_일시         36
인물:거주_국가         27
인물:거주_도시         26
단체:본사_주(도)       23
단체:정치/종교성향       21
단체:해산일           15
인물:종교            15
인물:출생_국가         13
인물:거주_주(도)       12
단체:주주            11
인물:용의자           11
인물:학교             9
단체:구성원_수          9
인물:사망_원인          7
인물:나이             5
단체:자회사            5
인물:출생_도시          4
인물:사망_국가          4
인물:사망_도시          3
Name: label, dtype: int64

In [192]:
combined_df.to_csv("/opt/ml/input/data/train/train_gold.tsv", sep='\t', index=False)

In [201]:
combine_df_read = pd.read_csv("/opt/ml/input/data/train/train_gold.tsv", delimiter='\t')
combined_df_read.head()

,Unnamed: 0,id,text,entity1,beg1,end1,entity2,beg2,end2,label,ans
0,0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30.0,33.0,자동차,19.0,21.0,단체:제작,NaN
1,1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5.0,7.0,27석,42.0,44.0,관계_없음,NaN
2,2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0.0,7.0,UEFA,9.0,12.0,단체:별칭,NaN
3,3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24.0,26.0,공격수,3.0,5.0,인물:직업/직함,NaN
4,4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0.0,2.0,퍼쿤 씨 인트라팃,32.0,40.0,인물:부모님,NaN


# Train-val split

In [ ]:
from sklearn.model_selection import 